<a href="https://colab.research.google.com/github/bluecityisu/class2022/blob/main/men_women_%E9%81%B7%E7%A7%BB%E5%BC%8F%E5%AD%B8%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

weights_url = "https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weights_file = "inception_v3.h5"
urllib.request.urlretrieve(weights_url, weights_file)

pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                                include_top=False,
                                weights=None)

pre_trained_model.load_weights(weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(2, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['acc'])

training_dir = r'/content/drive/MyDrive/class/202209/pic/train'

validation_dir = r'/content/drive/MyDrive/class/202209/pic/val'

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1.0/255.)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(training_dir,
                                                    batch_size=20,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode='categorical',
                                                         target_size=(150, 150))


history = model.fit_generator(
            train_generator,
            validation_data=validation_generator,
            epochs=10)

In [ ]:
print(len(model.layers))

In [ ]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
img_height=150
img_width=150

img_path = r'/content/drive/MyDrive/class/202209/pic/train/men/00000021.jpg'

img = image.load_img(img_path, target_size=(img_height, img_width))
img_tensor = image.img_to_array(img)
img_tensor1 = np.expand_dims(img_tensor, axis=0)
img_tensor1 = img_tensor1/255.0
img_tensor2 = np.vstack([img_tensor1])
#img_tensor3 = preprocess_input(img_tensor2)

featuremap = model.predict(img_tensor2)
plt.imshow(img)
#plt.imshow(featuremap)
print(np.around(featuremap,3))
print(featuremap.argmax())
#plt.imshow(img_tensor2[0])
#print (img_tensor.shape)

In [ ]:
#!pip install tensorflowjs
#import tensorflowjs as tfjs
#tfjs.converters.save_keras_model(model, 'models')

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x=x/255
  images = np.vstack([x])
  #images=preprocess_input(images)

  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes.argmax())

In [ ]:
#!zip -r /content/drive/MyDrive/class/202209 /content/models 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install tensorflowjs
import tensorflowjs as tfjs

In [ ]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, 'models')